In [1]:
# Import all the models and functions
from models import *

# Example of using a model

In [9]:
# Load data
topic_and_hardness = pd.read_json(
    "../data/training-set/chatbot-arena-gpt3-scores.jsonl.gz",
    lines=True,
    compression="gzip"
)
# Perform any feature engineering
pattern_z = r"\b\w+\b"
topic_and_hardness['prompt_length'] = topic_and_hardness['prompt'].apply(lambda x: len(re.findall(pattern_z, x)))
topic_and_hardness = topic_and_hardness.dropna()
# Create a model
mlr = MultiLinearRegressionModel(topic_and_hardness, regularization='l1')
# Preprocess the data with the features you want to use
mlr.preprocess(features=['prompt_length'])
# Train the model
mlr.train_model()
# Evaluate the model based on a prediction
mlr.evaluate(mlr.predict())

       prompt_length
8684              20
3273              13
12870              7
25162            100
17897             10
...              ...
21589             27
5392              43
861               25
15808              7
23668            177

[17679 rows x 1 columns]
3.086841757951696
